In [1]:
import pandas as pd
import os
from tqdm import tqdm
import numpy as np
import torch
from torch import optim
from torch import nn
from transformers import AutoModel, AutoTokenizer, AutoModelForSequenceClassification
from torch.utils.data import Dataset, DataLoader
from torch.nn import functional as F
from transformers import AutoTokenizer, AutoModelForSequenceClassification, BertConfig
from captum.attr import visualization as viz
from captum.attr import IntegratedGradients, LayerConductance, LayerIntegratedGradients
from captum.attr import configure_interpretable_embedding_layer, remove_interpretable_embedding_layer
import torch
import matplotlib.pyplot as plt
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
phobert = AutoModel.from_pretrained("vinai/phobert-base")
print(phobert)

# For transformers v4.x+: 
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base", use_fast=False)


ENTUBE = 'entube.parquet'
entube = pd.read_parquet(ENTUBE)
entube.head()

Some weights of the model checkpoint at vinai/phobert-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


RobertaModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(64001, 768, padding_idx=1)
    (position_embeddings): Embedding(258, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): RobertaEncoder(
    (layer): ModuleList(
      (0-11): 12 x RobertaLayer(
        (attention): RobertaAttention(
          (self): RobertaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): RobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dropou

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


,channel_id,channel_name,channel_category,channel_started,channel_rank,channel_subscribers,id,title,title_length,categories,...,comment_count,dislike_count,like_per_view,comment_per_view,dislike_per_view,engagement_rate_1,engagement_rate_2,q_score,label_1,label_2
0,UC0jDoh3tVXCaqJ6oTve8ebA,FAP TV,Comedy,2014,2,12800000,nehrVdADdH0,[FAP TV ] Thông Báo Tuyển Diễn Viên Nam Film L...,14,Film & Animation,...,827,105,0.020731,0.002456,0.000312,0.023187,0.023499,0.970364,2,2
1,UC0jDoh3tVXCaqJ6oTve8ebA,FAP TV,Comedy,2014,2,12800000,K66wOEaBwK4,Phía Sau Một Cô Gái - Soobin Hoàng Sơn | MV Fa...,14,Comedy,...,1594,664,0.010124,0.000622,0.000259,0.010746,0.011005,0.950070,1,2
2,UC0jDoh3tVXCaqJ6oTve8ebA,FAP TV,Comedy,2014,2,12800000,D00vn3X7oI8,FAPtv Cơm Nguội: Tập 94 - Dấu Ấn Học Đường Phần 2,12,Entertainment,...,2214,3089,0.006007,0.000234,0.000326,0.006241,0.006568,0.896920,1,2
3,UC0jDoh3tVXCaqJ6oTve8ebA,FAP TV,Comedy,2014,2,12800000,G22G1k3G-kM,FAPtv Cơm Nguội: Tập 100 - Hành Trình Vui Vẻ,10,Entertainment,...,1752,2202,0.007931,0.000228,0.000287,0.008159,0.008446,0.930209,1,2
4,UC0jDoh3tVXCaqJ6oTve8ebA,FAP TV,Comedy,2014,2,12800000,G5EG7ymPErw,FAPtv Cơm Nguội: Tập 95 - Dấu Ấn Học Đường Phầ...,12,Entertainment,...,2417,2208,0.006485,0.000303,0.000277,0.006789,0.007066,0.918029,1,2


In [2]:
def get_encode(text,s=100):
    temp=tokenizer.encode(text)
    while len(temp)<s:
        temp.append(1)
    input_ids = torch.tensor(temp[:s])
    return input_ids

def get_tags_encode(tags_list):
  return get_encode(" ".join([i for i in tags_list]))

In [3]:
entube_data = entube[['id','title','tags','label_2']]
entube_data

,id,title,tags,label_2
0,nehrVdADdH0,[FAP TV ] Thông Báo Tuyển Diễn Viên Nam Film L...,"[phim hai, FAPtv, FAPtivi, cơm nguội, Ribi Sac...",2
1,K66wOEaBwK4,Phía Sau Một Cô Gái - Soobin Hoàng Sơn | MV Fa...,"[phim hai, FAPtv, FAPtivi, cơm nguội, Ribi Sac...",2
2,D00vn3X7oI8,FAPtv Cơm Nguội: Tập 94 - Dấu Ấn Học Đường Phần 2,"[phim hai, FAPtv, FAPtivi, cơm nguội, Ribi Sac...",2
3,G22G1k3G-kM,FAPtv Cơm Nguội: Tập 100 - Hành Trình Vui Vẻ,"[phim hai, FAPtv, FAPtivi, cơm nguội, Ribi Sac...",2
4,G5EG7ymPErw,FAPtv Cơm Nguội: Tập 95 - Dấu Ấn Học Đường Phầ...,"[phim hai, FAPtv, FAPtivi, cơm nguội, Ribi Sac...",2
...,...,...,...,...
26231,3Dn6gHcMMxw,"Hot: Cha Đẻ ""ATM gạo"" Lần Đầu Ra Mắt ""ATM khẩu...","[Việt Nam Fly, Việt Nam, Fly, Việt Nam bay cao]",1
26232,GhxY-7QwD_c,"Đường Thành Sông, Nhà Thành Hầm Chứa Nước Sau ...","[Việt Nam Fly, Việt Nam, Fly, Việt Nam bay cao]",1
26233,MwqkdX9lyuk,"AEON MALL Bình Tân Quá Th.ê Thảm, Vắ.ng Hoe Vì...","[Việt Nam Fly, Việt Nam, Fly, Việt Nam bay cao]",1
26234,3dHgXfekVjo,"Sài Gòn Ngập Khắp Nơi Từ Chiều Đến Đêm, Ngang ...","[Việt Nam Fly, Việt Nam, Fly, Việt Nam bay cao]",1


In [4]:
entube_data['encode_title'] = entube_data['title'].apply(get_encode)
entube_data['encode_tag'] = entube_data['tags'].apply(get_tags_encode)
entube_data

C:\Users\mtthe\AppData\Local\Temp\ipykernel_19260\3004365453.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  entube_data['encode_title'] = entube_data['title'].apply(get_encode)
C:\Users\mtthe\AppData\Local\Temp\ipykernel_19260\3004365453.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  entube_data['encode_tag'] = entube_data['tags'].apply(get_tags_encode)


,id,title,tags,label_2,encode_title,encode_tag
0,nehrVdADdH0,[FAP TV ] Thông Báo Tuyển Diễn Viên Nam Film L...,"[phim hai, FAPtv, FAPtivi, cơm nguội, Ribi Sac...",2,"[tensor(0), tensor(63576), tensor(1579), tenso...","[tensor(0), tensor(266), tensor(82), tensor(15..."
1,K66wOEaBwK4,Phía Sau Một Cô Gái - Soobin Hoàng Sơn | MV Fa...,"[phim hai, FAPtv, FAPtivi, cơm nguội, Ribi Sac...",2,"[tensor(0), tensor(2696), tensor(162), tensor(...","[tensor(0), tensor(266), tensor(82), tensor(15..."
2,D00vn3X7oI8,FAPtv Cơm Nguội: Tập 94 - Dấu Ấn Học Đường Phần 2,"[phim hai, FAPtv, FAPtivi, cơm nguội, Ribi Sac...",2,"[tensor(0), tensor(1579), tensor(8607), tensor...","[tensor(0), tensor(266), tensor(82), tensor(15..."
3,G22G1k3G-kM,FAPtv Cơm Nguội: Tập 100 - Hành Trình Vui Vẻ,"[phim hai, FAPtv, FAPtivi, cơm nguội, Ribi Sac...",2,"[tensor(0), tensor(1579), tensor(8607), tensor...","[tensor(0), tensor(266), tensor(82), tensor(15..."
4,G5EG7ymPErw,FAPtv Cơm Nguội: Tập 95 - Dấu Ấn Học Đường Phầ...,"[phim hai, FAPtv, FAPtivi, cơm nguội, Ribi Sac...",2,"[tensor(0), tensor(1579), tensor(8607), tensor...","[tensor(0), tensor(266), tensor(82), tensor(15..."
...,...,...,...,...,...,...
26231,3Dn6gHcMMxw,"Hot: Cha Đẻ ""ATM gạo"" Lần Đầu Ra Mắt ""ATM khẩu...","[Việt Nam Fly, Việt Nam, Fly, Việt Nam bay cao]",1,"[tensor(0), tensor(16261), tensor(27), tensor(...","[tensor(0), tensor(350), tensor(590), tensor(4..."
26232,GhxY-7QwD_c,"Đường Thành Sông, Nhà Thành Hầm Chứa Nước Sau ...","[Việt Nam Fly, Việt Nam, Fly, Việt Nam bay cao]",1,"[tensor(0), tensor(2080), tensor(1206), tensor...","[tensor(0), tensor(350), tensor(590), tensor(4..."
26233,MwqkdX9lyuk,"AEON MALL Bình Tân Quá Th.ê Thảm, Vắ.ng Hoe Vì...","[Việt Nam Fly, Việt Nam, Fly, Việt Nam bay cao]",1,"[tensor(0), tensor(40989), tensor(11778), tens...","[tensor(0), tensor(350), tensor(590), tensor(4..."
26234,3dHgXfekVjo,"Sài Gòn Ngập Khắp Nơi Từ Chiều Đến Đêm, Ngang ...","[Việt Nam Fly, Việt Nam, Fly, Việt Nam bay cao]",1,"[tensor(0), tensor(10696), tensor(26328), tens...","[tensor(0), tensor(350), tensor(590), tensor(4..."


In [5]:
class SimplerModel(nn.Module):
    def __init__(self):
        super(SimplerModel, self).__init__()

        self.phobert_title = AutoModel.from_pretrained("vinai/phobert-base")
        self.title_LayerNorm = nn.LayerNorm(768)
        self.title_MultiheadAttention = nn.MultiheadAttention(768,1)
        self.title_Linear = nn.Linear(768, 1024)


        self.phobert_tag = AutoModel.from_pretrained("vinai/phobert-base")
        self.tag_LayerNorm = nn.LayerNorm(768)
        self.tag_MultiheadAttention = nn.MultiheadAttention(768,1)
        self.tag_Linear = nn.Linear(768, 1024)

        self.conv = nn.Sequential(
                nn.BatchNorm1d(2),
                nn.Conv1d(2, 1, 1),
                nn.ReLU(inplace=True)
            )
        self.cf = nn.Linear(1024,3)
    def forward(self, X):
        if (len(X.shape)==1):
            X = X[None,:]
        X_title = X[:,:100]
        X_tag = X[:,100:]

        with torch.no_grad():
            x_title = self.phobert_title(X_title)
        x_title = x_title['pooler_output']
        x_title_1 = x_title
        x_title = self.title_LayerNorm(x_title)
        x_title,_ = self.title_MultiheadAttention(x_title,x_title,x_title)
        x_title=x_title+x_title_1
        x_title = self.title_Linear(x_title)
        x_title = x_title[:,None,:]

        
        with torch.no_grad():
            x_tag = self.phobert_tag(X_tag)
        x_tag = x_tag['pooler_output']
        x_tag_1 = x_tag
        x_tag = self.tag_LayerNorm(x_tag)
        x_tag,_ = self.tag_MultiheadAttention(x_tag,x_tag,x_tag)
        x_tag=x_tag+x_tag_1
        x_tag = self.tag_Linear(x_tag)
        x_tag = x_tag[:,None,:]

        
        x = torch.concat((x_title, x_tag), dim=1)
        x = self.conv(x)
        x = F.dropout(x, 0.1)
        x = self.cf(x)
        return torch.squeeze(x)
simplermodel = SimplerModel()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using {device} device')
simplermodel = simplermodel.to(device)

Some weights of the model checkpoint at vinai/phobert-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at vinai/phobert-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.decoder.

Using cuda device


In [6]:
class CustomDataset(Dataset):
    def __init__(self, dataframe):
        self.dataframe = dataframe

    def __getitem__(self, index):
        row = self.dataframe.iloc[index].to_numpy()
        features =  torch.cat((row[4],row[5]),0)
        label = row[3]
        return features, label

    def __len__(self):
        return len(self.dataframe)
    
data = CustomDataset(dataframe=entube_data)
dataloader = DataLoader(data, batch_size = 20, drop_last=True,shuffle=True)
loss_fn = nn.CrossEntropyLoss()

In [7]:
train_size = int(0.8 * len(data))
test_size = len(data) - train_size
generator = torch.Generator().manual_seed(123)
train_loader, val_loader = torch.utils.data.random_split(data, [train_size, test_size],generator=generator)
trainloader = DataLoader(train_loader, batch_size=32)
validloader = DataLoader(val_loader, batch_size=32)

In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(simplermodel.parameters(), lr = 0.001)

In [9]:
PATH = "saved_model.pth"
simplermodel.load_state_dict(torch.load(PATH))

def valid(e,train_loss,min_valid_loss = np.inf):
    valid_loss = 0.0
    simplermodel.eval()
    for data, labels in validloader:
        if torch.cuda.is_available():
            data, labels = data.cuda(), labels.cuda()

        target = simplermodel(data)
        loss = criterion(target,labels)
        valid_loss+= loss.item()

    print(f'Epoch {e+1} \t\t Training Loss: {train_loss / len(trainloader)} \t\t Validation Loss: {valid_loss / len(validloader)}')
    if min_valid_loss > valid_loss:
        print(f'Validation Loss Decreased({min_valid_loss:.6f}--->{valid_loss:.6f}) \t Saving The Model')
        min_valid_loss = valid_loss
        torch.save(simplermodel.state_dict(), PATH)
    return min_valid_loss
def train():
    train_loss = 0.0
    simplermodel.train()
    for data, labels in tqdm(trainloader):
        if torch.cuda.is_available():
            data, labels = data.cuda(), labels.cuda()
        optimizer.zero_grad()
        target = simplermodel(data)
        loss = criterion(target,labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    return train_loss

In [10]:
epochs=0

min_valid_loss=valid(0,0)
for e in range(epochs):
    train_loss=train()
    min_valid_loss=valid(e,train_loss,min_valid_loss)

Epoch 1 		 Training Loss: 0.0 		 Validation Loss: 0.7885131308945214
Validation Loss Decreased(inf--->129.316153) 	 Saving The Model


In [46]:
simplermodel.phobert_title.embeddings

SimplerModel(
  (phobert_title): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(64001, 768, padding_idx=1)
      (position_embeddings): Embedding(258, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): 

In [12]:
def construct_input_ref_pair(text, ref_token_id, sep_token_id, cls_token_id):

    text_ids = tokenizer.encode(text, add_special_tokens=False)
    # construct input token ids
    input_ids = [cls_token_id] + text_ids + [sep_token_id]
    # construct reference token ids 
    ref_input_ids = [cls_token_id] + [ref_token_id] * len(text_ids) + [sep_token_id]

    return torch.tensor([input_ids], device=device), torch.tensor([ref_input_ids], device=device), len(text_ids)

def construct_input_ref_token_type_pair(input_ids, sep_ind=0):
    seq_len = input_ids.size(1)
    token_type_ids = torch.tensor([[0 if i <= sep_ind else 1 for i in range(seq_len)]], device=device)
    ref_token_type_ids = torch.zeros_like(token_type_ids, device=device)# * -1
    return token_type_ids, ref_token_type_ids

In [13]:
ref_token_id = tokenizer.pad_token_id # A token used for generating token reference
sep_token_id = tokenizer.sep_token_id # A token used as a separator between question and text and it is also added to the end of the text.
cls_token_id = tokenizer.cls_token_id

In [14]:

input_ids, ref_input_ids, sep_id = construct_input_ref_pair("Xuân Hinh Thanh Ngoan | Vợ Chồng Quán | Xuân Hinh Hat Chèo Hay Nhất", ref_token_id, sep_token_id, cls_token_id)

In [43]:
lig = LayerIntegratedGradients(custom_forward_0, simplermodel.phobert_title.embeddings)

In [44]:
attributions, delta = lig.attribute(inputs=input_ids,
                                    baselines=ref_input_ids,
                                    n_steps=700,
                                    internal_batch_size=32,
                                    return_convergence_delta=True)

IndexError: index 0 is out of bounds for dimension 1 with size 0

In [38]:
def predict(inputs):
    return simplermodel(inputs)

def custom_forward_0(inputs):
    preds = predict(inputs)
    return torch.softmax(preds, dim = 1)[:, 0]

In [42]:
for data, labels in validloader:
    if torch.cuda.is_available():
        data, labels = data.cuda(), labels.cuda()

    target = simplermodel(data)
    loss = criterion(target,labels)
    print(custom_forward_0(data).shape)
    break

torch.Size([32])
